# Demo Code for Predictions of Model Deeplearning_cnn
Running this code uses the model created in deeplearning_cnn notebook and predicts the test_potato dataset, the results is written in the csv file.

In [2]:
import os
import re
import torch
import pandas as pd
from PIL import Image
from torch import nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

# Class mapping
class_mapping = {
    0: 'Bacteria',
    1: 'Fungi',
    2: 'Healthy',
    3: 'Pest',
    4: 'Phytopthora',
    5: 'Virus'
}
num_classes = len(class_mapping)

class NumericallySortedImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Get files and sort numerically
        self.image_files = [f for f in os.listdir(root_dir) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Sort files numerically (extract numbers from filenames)
        def extract_number(f):
            s = re.findall(r'\d+', f)
            return int(s[0]) if s else float('inf')
            
        self.image_files.sort(key=extract_number)
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, self.image_files[idx]

def load_model(model_path, device):
    # Initialize ResNet50 model
    resnet = models.resnet50(pretrained=False)
    
    # Replace final fully connected layer
    resnet.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(resnet.fc.in_features, num_classes)
    )
    
    # Load saved weights
    resnet.load_state_dict(torch.load(model_path, map_location=device))
    resnet = resnet.to(device)
    resnet.eval()
    
    return resnet

def predict_unlabelled(model, data_loader, device):
    predictions = []
    with torch.no_grad():
        for images, filenames in data_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            for filename, pred in zip(filenames, preds.cpu().numpy()):
                predictions.append({
                    'image_name': filename,
                    'class': class_mapping[pred]
                })
    
    return predictions

# Main execution
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])

    image_dir = "potato_tests"  # Update this with your test directory
    model_path = "deeplearning_cnn.pt"  # Update this with your model path
    
    # Create numerically sorted dataset
    dataset = NumericallySortedImageDataset(image_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)

    model = load_model(model_path, device=device)
    predictions = predict_unlabelled(model, loader, device)

    # Create and save sorted DataFrame
    df = pd.DataFrame(predictions)
    
    # Add numerical index column extracted from filenames
    def extract_index(fname):
        nums = re.findall(r'\d+', fname)
        return int(nums[0]) if nums else 0
        
    df['file_index'] = df['image_name'].apply(extract_index)
    df = df.sort_values('file_index').drop('file_index', axis=1)
    
    csv_path = "deeplearning_cnn_predictions.csv"
    df.to_csv(csv_path, index=False)
    
    print(f"\nPredictions saved to {csv_path} (numerically ordered)")
    print("\nFirst 10 predictions:")
    print(df.head(10))

Using device: cuda


c:\Users\gerar\OneDrive\Documents\5 UP 3.2\CS 180\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gerar\OneDrive\Documents\5 UP 3.2\CS 180\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Predictions saved to deeplearning_cnn_predictions.csv (numerically ordered)

First 10 predictions:
  image_name        class
0     0.jpeg        Virus
1     1.jpeg  Phytopthora
2     2.jpeg         Pest
3     3.jpeg         Pest
4     4.jpeg        Fungi
5     5.jpeg         Pest
6     6.jpeg         Pest
7     7.jpeg      Healthy
8     8.jpeg        Virus
9     9.jpeg        Fungi
